In [54]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import gzip

# Pull all historical data

In [55]:
try:
    html = urlopen('https://gz.blockchair.com/bitcoin/blocks/?fbclid=IwAR2DtTOZeTeLCMVQC-TugYFBlDtFIijZGoSISG1RYbE5fmmGem15hpdCVo4')
except HTTPError as e:
    print(e)
except URLError as e:
    print('URLError: The server could not be found!')

In [56]:
blockchair_bs = BeautifulSoup(html, 'html.parser')

In [57]:
file_url_list = blockchair_bs.pre.find_all('a')

In [80]:
master_filepath = 'data/Blockchair/blockchair.tsv'

# i=0 does not refer to a valid link, so start from i=1
for i in range(1,len(file_url_list)):
    file_url = file_url_list[i].attrs['href']
    download_url = 'https://gz.blockchair.com/bitcoin/blocks/' + file_url
    
    # Download the file from `url` and save it locally under `filename`:
    filename = "data/Blockchair/" + file_url[:-3]
    
    with urlopen(download_url) as response, open(filename, 'wb') as file_out:
        file_out.write(gzip.decompress(response.read()))
    
    # Create master file at the first iteration
    if i == 1:
        with open(master_filepath, 'w') as final_tsv_file:
            with open(filename, 'r') as tsv_file:
                for row in tsv_file:
                    final_tsv_file.write(row)
    
    else:
        # append downloaded file to master file
        with open(master_filepath, 'a') as final_tsv_file: 
            with open(filename, 'r') as tsv_file:
                for row in tsv_file:
                    if row[:2] == 'id':
                        final_tsv_file.write('\n')
                    else:
                        final_tsv_file.write(row)

In [92]:
# Convert tsv file to csv
blockchair_df = pd.read_csv(master_filepath, sep='\t', 
                            dtype={'version_bits':'object','chainwork':'object'}, index_col=['id'])

In [93]:
blockchair_df.to_csv('data/Blockchair/blockchair.csv')

# Daily Update

In [103]:
try:
    html = urlopen('https://gz.blockchair.com/bitcoin/blocks/?fbclid=IwAR2DtTOZeTeLCMVQC-TugYFBlDtFIijZGoSISG1RYbE5fmmGem15hpdCVo4')
except HTTPError as e:
    print(e)
except URLError as e:
    print('URLError: The server could not be found!')

blockchair_bs = BeautifulSoup(html, 'html.parser')
file_url_list = blockchair_bs.pre.find_all('a')

In [104]:
file_url = file_url_list[-1].attrs['href']
download_url = 'https://gz.blockchair.com/bitcoin/blocks/' + file_url

# Download the file from `url` and save it locally under `filename`:
filename = "data/Blockchair/" + file_url[:-3]

with urlopen(download_url) as response, open(filename, 'wb') as file_out:
    file_out.write(gzip.decompress(response.read()))

# Convert to csv before appending
new_filename = filename[:-3]+'csv'
blockchair_df = pd.read_csv('data/Blockchair/blockchair'+filename, sep='\t')
blockchair_df.to_csv(new_filename)
    
# Append converted file to master file
with open('data/Blockchair/blockchair.csv', 'a') as final_csv_file: 
    with open(new_filename, 'r') as csv_file:
        for row in csv_file:
            final_csv_file.write(row)

FileNotFoundError: [Errno 2] File b'data/Blockchair/blockchairdata/Blockchair/blockchair_bitcoin_blocks_20191003.tsv' does not exist: b'data/Blockchair/blockchairdata/Blockchair/blockchair_bitcoin_blocks_20191003.tsv'

# Check

In [99]:
# Check if there are duplicated ids
assert blockchair_df.index.duplicated().sum() == 0